In [ ]:
import sys
#Fix problem with ros python path
sys.path.remove('/opt/ros/kinetic/lib/python2.7/dist-packages')

In [ ]:
import numpy as np
import os, math
import pascal_voc_writer
from tqdm import tqdm
import cv2
import csv
from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
# Old methods
def generateMaskColor(frame, frameNum):
    frameStr = "frame"+str(frameNum).zfill(6)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    (b, g, r) = cv2.split(frame)
    ret, b = cv2.threshold(b, 200, 255, cv2.THRESH_BINARY)
    ret, r = cv2.threshold(r, 254, 255, cv2.THRESH_BINARY)
    rb = cv2.bitwise_and(r,b)
    cv2.imwrite(out_path+"Orientation/"+frameStr+".jpg", rb)
    #cv2.imshow('orig',rb)
def generateMaskHSV(frame):
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    (h, s, v) = cv2.split(frame)
    cv2.imshow('orig',v)  

In [ ]:
def centroid(contours):
    if len(contours) > 0:
        cnt = contours[0]    
        M = cv2.moments(cnt)
        cx = int(M['m10']/M['m00'])
        cy = int(M['m01']/M['m00'])
        return cx,cy
    return 0,0

def minRect(frame, frameNum):
    frameStr = "frame"+str(frameNum).zfill(6)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))
    (b, g, r) = cv2.split(frame)
    ret, b = cv2.threshold(b, 200, 255, cv2.THRESH_BINARY)
    b = cv2.dilate(b,kernel1)
    
    thetas = []
    im2, contours, hierarchy = cv2.findContours(b,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    if len(contours) > 0:
        rect = cv2.minAreaRect(contours[0])
        box = cv2.boxPoints(rect)
        box = np.int0(box)
        for j in range(4):
            thetas.append(math.atan2(box[(j+1)%4][0]-box[j][0],box[(j+1)%4][1]-box[j][1]) * 180 / np.pi)
        if PREPARATION_MODE:
            cv2.drawContours(frame,[box],0,(0,255,255),2)
    return centroid(contours), thetas
    
def minRectR(frame, frameNum):
    frameStr = "frame"+str(frameNum).zfill(6)
    kernel1 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
    kernel2 = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(50,50))
    (b, g, r) = cv2.split(frame)
    ret, b = cv2.threshold(b, 200, 255, cv2.THRESH_BINARY)
    ret, r = cv2.threshold(r, 240, 255, cv2.THRESH_BINARY)
    r = cv2.erode(r,kernel1)
    r = cv2.dilate(r,kernel2)
    
    im2, contours, hierarchy = cv2.findContours(r,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
    return centroid(contours)

In [ ]:
path = "/media/data/LocalizationData/"
out_path = path+"Output/Validation/CoptersFlying1/"
os.makedirs(out_path+"Orientation", exist_ok=True)
PREPARATION_MODE = False

cap2 = cv2.VideoCapture('/media/data/LocalizationData/Videos/CopterWithLEDFlight/copters_flying1.avi')

csv_rows = []
for i in tqdm(range(int(cap2.get(cv2.CAP_PROP_FRAME_COUNT)))):
    ret, frame = cap2.read()
    if ret == False:
        break
    if i%4 == 0:
        cx2, cy2 = minRectR(frame,i)
        (cx, cy), thetasRect = minRect(frame,i)
        frameStr = "frame"+str(i).zfill(6)
        theta = math.atan2(cx2-cx,cy2-cy) * 180 / np.pi
        for t in thetasRect:
            if min(abs(t-theta),360-abs(t-theta)) < 45:
                theta = t
        #print(theta)
        
        # TODO: Use rotated rect for theta
        
        if PREPARATION_MODE:
            size = 100
            cv2.circle(frame,(cx2,cy2), 5, (0,0,255), -1)
            cv2.circle(frame,(cx,cy), 5, (0,255,0), -1)
            cv2.arrowedLine(frame,
                (cx,cy),
                (int(cx-math.sin(theta*np.pi/180)*size),int(cy-math.cos(theta*np.pi/180)*size)), 
                (0,0,255),3)
            cv2.imshow('outframe', frame)
        else:
            file_name = out_path+"Orientation/"+frameStr+".jpg"
            csv_rows.append([file_name,theta,cx,cy])
            cv2.imwrite(file_name, frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

with open(out_path+'Orientation/orientations.csv', 'w') as csvfile:
    spamwriter = csv.writer(csvfile, delimiter=',',
                            quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for row in csv_rows:
        spamwriter.writerow(row)
        
cap2.release()
cv2.destroyAllWindows()